In [1]:
import os
import pandas as pd

# xlsx 파일들이 들어있는 디렉토리 경로
directory_path = "Z:/02.펀드/018. 성과측정/2023/2023H1/주식매매"

# 디렉토리 내의 모든 xlsx 파일을 읽어들이기
file_list = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.xlsx')]

# 파일을 읽어들여 데이터프레임으로 변환
dataframes = []
for file in file_list:
    # 첫 번째 행을 인덱스로 설정하고자 함
    df = pd.read_excel(file, index_col=0)
    dataframes.append(df)

# 데이터프레임 병합
result = pd.concat(dataframes)

print(result)

# 펀드코드' 칼럼이 비어있는 행 삭제
result = result.dropna(subset=['펀드코드'])

# 결과를 Excel 파일로 저장
result_excel_path = os.path.join(directory_path, 'merged_stock_trade.xlsx')
result.to_excel(result_excel_path, encoding='utf-8')

print(f"병합된 결과가 {result_excel_path}에 저장되었습니다.")

           구분 거래구분        거래일     펀드코드                  펀드명          종목코드  \
No                                                                          
1    집합투자증권매수  NaN        NaT      NaN                  NaN           NaN   
2    집합투자증권매수   일반 2023-01-02  DM12001    두나미스 공모주 일반사모투자신탁  KR7251340006   
3    집합투자증권매수   일반 2023-01-02  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7229200001   
4    집합투자증권매수   일반 2023-01-02  DM13001  두나미스 코스닥밴처 일반사모투자신탁  KR7252670005   
5    집합투자증권매수   일반 2023-01-02  DM14001   두나미스 멀티전략 일반사모투자신탁  KR7229200001   
..        ...  ...        ...      ...                  ...           ...   
918      주식매도   차입 2023-06-30  DM14001   두나미스 멀티전략 일반사모투자신탁  KR7298020009   
919      주식매도   차입 2023-06-30  DM14001   두나미스 멀티전략 일반사모투자신탁  KR7393890009   
920      주식매도   일반 2023-06-30  DM15002         두나미스인베스트먼트홀딩  KR7340810001   
921      주식매도   일반 2023-06-30  DM15002         두나미스인베스트먼트홀딩  KR7354320004   
922      주식매도   일반 2023-06-30  DM15004      주식회사두나미스파트너스_NH  KR7354320004   

c:\Users\jtlee\AppData\Local\Programs\Python\Python39-32\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


병합된 결과가 Z:/02.펀드/018. 성과측정/2023/2023H1/주식매매\merged_stock_trade.xlsx에 저장되었습니다.


##### 당기 주식평가손익 계산 방법
1. 당기에 결산이 없는 경우
- 기말 주식평가손익 - 기초 주식평가손익
2. 중간에 결산이 있는 경우
- 기말 주식평가손익 + (결산일 주식평가손익 - 기초 주식평가손익)
3. 유의할 점
- 기말에 남아있는 종목에 대해서만 평가손익 계산할 것.
- 공모주는 무조건 대표님 포지션
--> 어떻게 구분? 수기로 하는 수밖에
##### 필요한 정보
1) rawdata 파일제목에 들어가야할 정보
- 일자구분(기초, 기말, 결산), (펀드코드)
2) rawdata에서 매칭시켜야 할 정보
- 펀드 : 펀드코드 또는 펀드명으로 매칭
- 종목 : 종목코드(티커)로 매칭.(종목명은 트레이딩 팀에서 임의로 변경하는 경우에 있기에 불가)
- 매니저 : 각 종목이 어떤 매니저의 포지션인지 구분한 후 매칭